<a href="https://colab.research.google.com/github/JoaquinBarrera/03MAIR---Algoritmos-de-optimizaci-n--2021/blob/main/Prueba_Confianza_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import pandas as pd
from pathlib import Path
import numpy as np
pd.options.display.float_format = '{:,.2f}'.format

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
root = Path('/content/drive/MyDrive/Prueba')

In [26]:
#Leer tabla de siniestros
siniestros = pd.read_csv(root / 'Siniestros.csv', parse_dates=['FECEXP','FECINI', 'FECTER', 'FECAVI', 'FECOCU'], dtype={'SINIESTRO': str, 'POLIZA':str, 'TOTAL_BRUTO':float}, sep=';')
# Trasnformar Total neto a valores flotantes
siniestros["TOTAL_NETO"] = pd.to_numeric([float(str(i).replace(",", ".")) for i in siniestros["TOTAL_NETO"]])
siniestros.drop(['Unnamed: 18'], axis=1, inplace=True)
siniestros.head(5)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,RAMO,SUC,SINIESTRO,POLIZA,FECEXP,FECINI,FECTER,FECAVI,FECOCU,TOTAL_BRUTO,TOTAL_NETO,REASEGURO,CORTE,MES,ORIGEN,AY,UY,CATEGORIA_PRODUCTO
0,2,101,671,898566,1998-04-12,1998-03-12,2001-03-11,2003-12-30,2000-03-21,"6,516,350.00","9,841,608.33",0,2008,1,PAGADOS,2000,"1,998.00",RC DERIVADO CUMPLIMIENTO
1,2,106,737,4020085,1994-05-10,1994-01-06,1995-01-06,2005-07-18,1994-10-09,"5,341,081.00","8,034,161.50",0,2008,1,PAGADOS,1994,"1,994.00",RC DERIVADO CUMPLIMIENTO
2,2,107,632,1270956,2001-08-30,2001-08-30,2002-10-06,2001-11-20,2001-11-20,"2,122,818.00","3,203,102.49",0,2008,1,PAGADOS,2001,"2,001.00",RC DERIVADO CUMPLIMIENTO
3,2,123,RO875,RO002215,2005-05-18,2005-05-18,2005-08-18,2007-03-16,2005-07-17,"8,710,016.00","13,066,557.33",0,2008,1,PAGADOS,2005,"2,005.00",RC DERIVADO CUMPLIMIENTO
4,2,123,RO875,RO002215,2005-05-18,2005-05-18,2005-08-18,2007-03-16,2005-07-17,"9,013,829.00","13,526,232.83",0,2008,1,PAGADOS,2005,"2,005.00",RC DERIVADO CUMPLIMIENTO


In [27]:
# Leer tabla de producción
produccion = pd.read_csv(root / 'produccion.txt', parse_dates=["FECINIP", "FECFIN", "FECEXP"], dtype={'POLIZA':str}, sep='|')
# Arreglar prima y valores asegurados que esta como string
produccion["Prima"] = pd.to_numeric([float(str(i).replace(",", ".")) for i in produccion["PRIMA_BRUTA"]])
produccion["Valor asegurado"] = pd.to_numeric([float(str(i).replace(",", ".")) for i in produccion["VASEG_BRUTO"]])
produccion.head(5)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,SUCUR,POLIZA,FECINIP,FECFIN,FECEXP,PRIMA_BRUTA,VASEG_BRUTO,Prima,Valor asegurado
0,101,1097263,2000-04-01,2001-03-31,2000-04-14,933466,"155577679,2","933,466.00","155,577,679.20"
1,101,1097263,2000-04-01,2001-03-31,2000-04-25,0,"155577679,2",0.00,"155,577,679.20"
2,101,1097263,2000-04-01,2001-12-31,2001-01-18,720520,169190726,"720,520.00","169,190,726.00"
3,101,1097263,2000-04-01,2001-12-31,2001-01-22,0,169190726,0.00,"169,190,726.00"
4,101,1097263,2000-04-01,2002-12-31,2002-01-10,765000,182133817,"765,000.00","182,133,817.00"


In [28]:
# Extraer año UY
produccion['Año Suscripción'] = produccion['FECEXP'].dt.year 

#Calcular Tasa Comercial Observada por año antes de cruzar las tablas
tco = produccion.groupby('Año Suscripción').sum()
tco['Tasa Comercial Observada 2 (%)'] = tco['Prima'] / tco['Valor asegurado']

In [29]:
# Cruzar tablas con metodo Inner (default)
df = pd.merge(produccion, siniestros, left_on=['SUCUR', 'POLIZA'], right_on=['SUC', 'POLIZA'], how='inner')

#Agrupar por año
df_final = df.groupby(['Año Suscripción']).agg({'Prima':'sum', 'Valor asegurado':'sum','TOTAL_BRUTO':'sum', 'TOTAL_NETO':'sum'})

# Calcular siniestralidad y tasa comercial observada
df_final['Siniestralidad (%)'] = df_final['TOTAL_BRUTO']/ df_final['Prima']
df_final['Tasa Comercial Observada (%)'] = df_final['Prima']/ df_final['Valor asegurado']

#Añadir tasa comercial observada calculada antes de haber cruzado las tablas
df_final = df_final.join(tco['Tasa Comercial Observada 2 (%)'])

In [30]:
df_final.rename(columns={'TOTAL_BRUTO':'Pagos'}, inplace=True)
df_final = df_final[['Prima', 'Valor asegurado', 'Pagos', 'Siniestralidad (%)',  'Tasa Comercial Observada (%)', 'Tasa Comercial Observada 2 (%)']]
df_final

,Prima,Valor asegurado,Pagos,Siniestralidad (%),Tasa Comercial Observada (%),Tasa Comercial Observada 2 (%)
Año Suscripción,,,,,,
1993,"49,706,185.00","19,000,000,000.00","4,609,762,626.00",92.74,0.00,0.00
1994,"2,544,703,174.00","104,660,609,280.00","2,889,921,194.00",1.14,0.02,0.03
1995,"325,449,600.00","291,200,000,000.00","1,529,020,703.00",4.70,0.00,0.01
1996,"1,082,191,229.00","447,600,938,782.00","2,827,048,165.00",2.61,0.00,0.00
1997,"2,929,098,546.00","624,113,017,777.40","7,344,509,215.00",2.51,0.00,0.00
1998,"5,490,062,973.00","1,045,209,073,912.40","31,637,935,454.00",5.76,0.01,0.00
1999,"9,321,025,230.00","2,557,936,255,095.20","30,441,664,450.00",3.27,0.00,0.00
2000,"13,180,024,069.90","5,698,740,742,564.00","56,982,752,873.00",4.32,0.00,0.00
2001,"36,968,437,825.00","11,562,827,562,496.50","109,811,640,397.00",2.97,0.00,0.00


In [31]:
df_final.to_csv(root/ 'resultado.csv')